# Driving a Financial Valuation from a Design

As seen in
[Loading a Design](load_design.ipynb), a design's object model (a
[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph) including all
its spatial and semantic relationships) can be loaded via Speckle from any of
Speckle's supported [connectors](https://speckle.guide/user/connectors.html), so
long as those objects (`Entity`s and `Relationship`s) have been assigned.

In this notebook, we will demonstrate how to integrate financial modelling into
the object model (graph) in order to derive the financial valuation of a real
estate property from its spatial design.
This is done by calculating cash flows per `Entity`, and then using the
`Relationship`s to aggregate those `Flow`s into appropriate `Stream`s.

## Load the Design
We will use the same design as in the previous notebook.

In [1]:
import os

import locale
import urllib.parse
import json
import pandas as pd
import networkx as nx
from IPython.display import IFrame
import plotly.subplots
import plotly.offline as py

from specklepy.api import operations, client
from specklepy.transports.server import ServerTransport

# Load environment variables from .env file
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

import rangekeeper as rk

In [2]:
speckle = client.SpeckleClient(host="app.speckle.systems")
speckle.authenticate_with_token(os.getenv("SPECKLE_TOKEN"))

print(f"\n✓ Authenticated as: {speckle.account.userInfo.name}")

# Explore the Speckle Project, Model, and Versions
project = speckle.project.get_with_models("c0f66c35e3")
print(f"Project: {project.name}, ID: {project.id}")
print(
    f"Models: {[model.name for model in project.models.items]}, {[model.id for model in project.models.items]}"
)

model = speckle.model.get_with_versions(
    project_id=project.id,
    model_id="119ad04487",
)
print(
    f"Model: {model.name}, Versions: {[version.id for version in model.versions.items]}"
)
IFrame(f"https://app.speckle.systems/projects/{project.id}/models/{model.id}", width='100%', height=800)


✓ Authenticated as: Daniel Fink
Project: RangekeeperDemo, ID: c0f66c35e3
Models: ['main'], ['119ad04487']
Model: main, Versions: ['9a9670946f']


### Inspect the Model Graph

In [3]:
# Load the root Speckle Object of the design
transport = ServerTransport(stream_id="c0f66c35e3", client=speckle)
root = operations.receive(
    obj_id="e7acaac21ae7e9369339900a4aaeb827",
    remote_transport=transport,
)
parsed = rk.api.Speckle.parse(base=root['@property'])
property = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='property',
    type='archetype')

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


In [4]:
property.plot(name=f'assets/{property.name}')
IFrame(src=f'./{property.name}.html', width='100%', height=800)

assets/property.html


## Assigning and Aggregating `Flow`s per `Entity`
Now we can organise the two main categories of cash flows (costs and revenues)
according to the `Relationship` types in the design.

In this example, revenue-based cash flows are generated by `Entity`s that have a
measureable floor area -- in this case, anything that has a 'gfa' property.

We can query the graph to find them:

### Revenue-producing Entities
Revenue-producing `Entity`s will be aggregated by their spatial containment. So
we can first slice our graph to a tree of nodes that have "contains"
relationships:

In [5]:
spatial_containment = property.filter_by_type(
    relationship_type='spatiallyContains',
    name='spatial_containment')

Note: we anticipate that the spatial decomposition is completely hierarchical
(i.e., no spaces overlap, which means no spaces have multiple parents). We can
check this by testing whether the containment graph is a "tree" (or
"arborescence": see [NetworkX Tree](https://networkx.org/documentation/stable/reference/algorithms/tree.html#tree)).

In [6]:
nx.is_arborescence(spatial_containment.graph)

True

In [7]:
# Plot the spatial containment graph
spatial_containment.plot(name=f'assets/{spatial_containment.name}')
IFrame(src=f'./{spatial_containment.name}.html', width='100%', height=800)

assets/property by spatiallyContains and None.html


#### Simple Aggregation
Now that we have verified this, we can continue with a simple aggregation. We
wish to aggregate up through our hierarchy all `Entity`s that have a Gross Floor
Area ('gfa') property. *Rangekeeper* provides defaults for the `Entity.aggregate()`
method for this, whereby a named property of any `Entity` is summed (numerical
addition) into its parent `Entity` against a specified label:

In [8]:
property.aggregate(
    property='gfa',
    label='subtotal_gfa',
    relationship_type='spatiallyContains')

We can tabulate this by converting the graph to a pandas `DataFrame`. Note that
the graph nodes (as witnessed in the DataFrame's index and 'parent' column)
are GUID strings. Each of these have corresponding `Entity` objects, organised
via NetworkX's Node Attributes. See [NetworkX Tutorial](https://networkx.org/documentation/stable/tutorial.html#what-to-use-as-nodes-and-edges)

In [9]:
df = spatial_containment.to_DataFrame()
df = df[['name', 'type', 'gfa', 'subtotal_gfa', 'parent', 'use', 'ffl', 'number']]
df

,name,type,gfa,subtotal_gfa,parent,use,ffl,number
afe8e730-ad0f-4c2a-a15c-e249e4518fa8,property,property,NaN,42786.259115,None,NaN,NaN,NaN
0a0a97ef-431d-471d-bc33-ace7e7df5fcb,plinth,building,NaN,12773.744211,afe8e730-ad0f-4c2a-a15c-e249e4518fa8,NaN,NaN,NaN
5ebb9736-5f18-4103-a5a4-040cd2e9b019,buildingA,building,NaN,16575.630074,afe8e730-ad0f-4c2a-a15c-e249e4518fa8,NaN,NaN,NaN
a1e00481-e51e-47c5-b629-8b214c0900bd,buildingB,building,NaN,13436.884829,afe8e730-ad0f-4c2a-a15c-e249e4518fa8,NaN,NaN,NaN
6496e279-88ca-49f4-b48c-6c0f156e0a58,utilities,utilities,NaN,0.000000,afe8e730-ad0f-4c2a-a15c-e249e4518fa8,NaN,NaN,NaN
2b51847e-ec54-415d-afa0-ce32d74c6144,plinthparking,space,NaN,12773.744211,0a0a97ef-431d-471d-bc33-ace7e7df5fcb,parking,NaN,NaN
938f4cf1-d307-4160-9538-d4bfa9583ad8,plinthplant,utilities,NaN,0.000000,0a0a97ef-431d-471d-bc33-ace7e7df5fcb,plant,NaN,NaN
e84b8c35-fd4a-49fc-b4c9-e318a958685b,plinthparkingFloor-2,floor,6386.872106,6386.872106,2b51847e-ec54-415d-afa0-ce32d74c6144,NaN,-5.9,-2.0
c3ebe151-1e0c-45c6-8852-7ea21f72290e,plinthparkingFloor-1,floor,6386.872106,6386.872106,2b51847e-ec54-415d-afa0-ce32d74c6144,NaN,-2.8,-1.0
c0348c0f-a31f-421b-b6bc-0fe0d5ea947f,buildingAhotel,space,NaN,11716.690054,5ebb9736-5f18-4103-a5a4-040cd2e9b019,hotel,NaN,NaN


We can also plot this as a hierarchical pie chart (a.k.a. 'sunburst' chart), or
a treemap:

In [10]:
sunburst = spatial_containment.sunburst(property='subtotal_gfa')
treemap = spatial_containment.treemap(property='subtotal_gfa')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]],
    subplot_titles=('Gross Floor Area - Sunburst Plot', 'Gross Floor Area - Treemap Plot'))
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='spatial_containment_chart.html'

py.plot(fig, filename=f'assets/{filename}', auto_open=False)
IFrame(src=f'./{filename}', width='100%', height=800 * len(fig.data))

#### Aggregating a (Financial) Calculation
A more complex aggregation involves collecting specific cash `Flow`s into
`Stream`s per `Entity`/`Assembly`; for instance, we may want to aggregate all
revenues generated by lettable floorspace into their respective parent `Entity`s,
in order to analyse or compare the performance of the design at different
resolutions.

We start by identifying which `Entity`s are 'floor' types and have an area
measurement:

In [11]:
floors = property.filter_by_type(entity_type='floor')
floors = [floor for floor in floors.get_entities().values() if hasattr(floor, 'gfa')]

For each floor we can project a set of simple cash flows, in a similar manner
to [A Basic DCF Valuation](basic_dcf.ipynb).

First initialize our locale, and a set of parameters, including some around
area efficiency ratios, initial incomes per area, and growth rates:

In [12]:
locale.setlocale(locale.LC_ALL, 'en_au')
units = rk.measure.Index.registry
currency = rk.measure.register_currency(registry=units)
frequency = rk.duration.Type.YEAR

In [13]:
efficiency_ratios = dict(
    hotel=0.8,
    retail=0.675,
    residential=0.75,
    parking=0.9
    )
initial_income_per_area_pa = dict(
    hotel=750,
    retail=1000,
    residential=600,
    parking=0
    )
pgi_growth_rates = dict(
    hotel=.035,
    retail=.075,
    residential=.055,
    parking=0
    )
vacancy_rates = dict(
    hotel=.025,
    retail=.075,
    residential=.015,
    parking=0,
    )

In [14]:
revenue_inputs = pd.DataFrame(
    data=dict(
        efficiency_ratio=efficiency_ratios,
        initial_income_per_area_pa=initial_income_per_area_pa,
        pgi_growth_rate=pgi_growth_rates,
        vacancy_rate=vacancy_rates,
        ))
revenue_inputs

,efficiency_ratio,initial_income_per_area_pa,pgi_growth_rate,vacancy_rate
hotel,0.800,750,0.035,0.025
retail,0.675,1000,0.075,0.075
residential,0.750,600,0.055,0.015
parking,0.900,0,0.000,0.000


In [15]:
params = dict(
    start_date=pd.Timestamp('2001-01-01'),
    num_periods=10,
    frequency=rk.duration.Type.YEAR
    )

##### `Stream`s of Revenue `Flow`s per `Entity`:
We can now construct a set of cash flows per `Entity` in our design for our
asset. We will use a similar structure to the previous notebooks.

First we will define a general 'Spans' class that defines the time periods. This
can be used by any subsequent calculations:

In [16]:
class Spans:
    def __init__(self, params: dict):
        self.params = params
        self.calc_span = rk.duration.Span.from_duration(
            name='Span to Calculate Reversion',
            date=self.params['start_date'],
            duration=self.params['frequency'],
            amount=self.params['num_periods'] + 1)
        self.acq_span = rk.duration.Span.from_duration(
            name='Acquisition Span',
            date=rk.duration.offset(
                self.params['start_date'],
                amount=-1,
                duration=self.params['frequency']),
            duration=self.params['frequency'])
        self.span = self.calc_span.extend(
            name='Span',
            amount=-1,
            duration=self.params['frequency'])

Then, we will define a `Revenues` class that holds the parameters:

In [17]:
class Revenues:
    def __init__(
            self,
            params: dict,
            spans: Spans):
        self.params = params
        self.spans = spans

And then we add Revenue `Flow`s:

In [18]:
@rk.update_class(Revenues)
class Revenues:
    def generate(self):
        self.pgi = rk.flux.Flow.from_projection(
            name='Potential Gross Income',
            value=self.params['initial_income'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['pgi_growth_rate']),
            sequence=self.spans.calc_span.to_sequence(frequency=self.params['frequency'])),
            units=currency.units)
        self.vacancy = rk.flux.Flow(
            name='Vacancy Allowance',
            movements=self.pgi.movements * -self.params['vacancy_rate'],
            units=currency.units)
        self.egi = rk.flux.Stream(
            name='Effective Gross Income',
            flows=[self.pgi, self.vacancy],
            frequency=self.params['frequency'])

Now we need to add `Entity`-specific parameters to the parameters dictionary,
and run  the `init_spans()` and `init_flows()` methods per floor `Entity`:

(Note we place the resultant `Stream` in the `events` attribute of the `Entity`)

In [19]:
spans = Spans(params)

In [20]:
for floor in floors:
    parent = floor.get_relatives(
        relationship_type='spatiallyContains',
        outgoing=False,
        assembly=spatial_containment)[0]
    floor['use'] = parent['use']

    floor.params = params.copy()
    floor.params['initial_income'] = initial_income_per_area_pa[floor['use']] * floor['gfa'] * efficiency_ratios[floor['use']]
    floor.params['pgi_growth_rate'] = pgi_growth_rates[floor['use']]
    floor.params['vacancy_rate'] = vacancy_rates[floor['use']]

    revenues = Revenues(floor.params, spans)
    revenues.generate()
    floor['pgi'] = revenues.pgi
    floor['vacancy'] = revenues.vacancy
    floor['egi'] = revenues.egi

##### Aggregating by Parent Container
Since we have the "containment" hierarchy, we can aggregate the `Revenues` per
parent `Entity`:

*Rangekeeper* provides some helper methods to assist in the aggregation of
`Stream`s and `Flow`s; specifically, since the `Entity.aggregate()` method
updates a specified `Entity`'s property with a dictionary of subentity
properties, keyed by their respective `entityId`s, we can use the
`aggregate_flows()` function to support the collation of child `Entity`
`Flow`s into a single `Stream` in a new property label in the `Entity`

In [21]:
def aggregate_egis(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Effective Gross Income',
        frequency=frequency)

This function is passed into the `Entity.aggregate()` method as the `function`
parameter:

In [22]:
property.aggregate(
    function=aggregate_egis,
    property='egi',
    label='subtotal_egi',
    relationship_type='spatiallyContains')

Now we can look at what the `aggregate` method has produced for a single floor:

In [23]:
floors[1]['egi'].name

'Effective Gross Income'

We can then inspect the `aggregate_egi` property of an `Entity`, which holds a
`Stream` of `Flow`s (each coming from a child of that `Entity`). Let's use the
hotel compartment in Building A:

In [24]:
buildingAhotel = [e for (id, e) in spatial_containment.get_entities().items() if e.name == 'buildingAhotel'][0]
print(buildingAhotel.name)
buildingAhotel['subtotal_egi']

buildingAhotel


,Effective Gross Income for bc265fcf-d0f6-4d22-8e92-d8f86f4d707a [buildingAhotelFloor0],Effective Gross Income for 22851887-f9ee-4aa8-8755-bc021c3a3b30 [buildingAhotelFloor1],Effective Gross Income for d67cc046-9c03-4e43-b1b5-876417dbcdda [buildingAhotelFloor2],Effective Gross Income for 5f90565d-44c0-4de2-b510-d263599953f5 [buildingAhotelFloor3],Effective Gross Income for f7b76e8b-93f0-47c6-8666-05c541c19e25 [buildingAhotelFloor4],Effective Gross Income for 9bdec185-a51b-4845-b746-ddcd3e3fa433 [buildingAhotelFloor5],Effective Gross Income for 250edc28-32de-4633-add0-a384305b5e87 [buildingAhotelFloor6],Effective Gross Income for e4f25e01-0e9d-4fd2-b705-96d118802468 [buildingAhotelFloor7],Effective Gross Income for 9582fd3c-ce66-42c7-8675-39b04b7a4689 [buildingAhotelFloor8],Effective Gross Income for 9151983a-1344-4c88-81f3-76cd4ca8ec6d [buildingAhotelFloor9],Effective Gross Income for 56eec388-d1bb-4de7-92a8-be8202d5be2d [buildingAhotelFloor10],Effective Gross Income for 49d884ea-f590-46a3-a662-db5e73190848 [buildingAhotelFloor11],Effective Gross Income for 192fb1b2-fc6c-4b42-8135-45cbcbead777 [buildingAhotelFloor12],Effective Gross Income for 85305abb-0e6f-4c8f-bf27-553292e317e9 [buildingAhotelFloor13]
2001,"$65,338.65","$993,009.17","$743,025.13","$624,901.87","$442,798.89","$442,798.89","$442,798.89","$442,798.89","$442,798.89","$442,798.89","$442,798.89","$442,798.89","$442,798.89","$442,798.89"
2002,"$67,625.50","$1,027,764.49","$769,031.00","$646,773.44","$458,296.85","$458,296.85","$458,296.85","$458,296.85","$458,296.85","$458,296.85","$458,296.85","$458,296.85","$458,296.85","$458,296.85"
2003,"$69,992.40","$1,063,736.25","$795,947.09","$669,410.51","$474,337.24","$474,337.24","$474,337.24","$474,337.24","$474,337.24","$474,337.24","$474,337.24","$474,337.24","$474,337.24","$474,337.24"
2004,"$72,442.13","$1,100,967.02","$823,805.24","$692,839.87","$490,939.04","$490,939.04","$490,939.04","$490,939.04","$490,939.04","$490,939.04","$490,939.04","$490,939.04","$490,939.04","$490,939.04"
2005,"$74,977.60","$1,139,500.86","$852,638.42","$717,089.27","$508,121.91","$508,121.91","$508,121.91","$508,121.91","$508,121.91","$508,121.91","$508,121.91","$508,121.91","$508,121.91","$508,121.91"
2006,"$77,601.82","$1,179,383.39","$882,480.77","$742,187.39","$525,906.17","$525,906.17","$525,906.17","$525,906.17","$525,906.17","$525,906.17","$525,906.17","$525,906.17","$525,906.17","$525,906.17"
2007,"$80,317.88","$1,220,661.81","$913,367.59","$768,163.95","$544,312.89","$544,312.89","$544,312.89","$544,312.89","$544,312.89","$544,312.89","$544,312.89","$544,312.89","$544,312.89","$544,312.89"
2008,"$83,129.01","$1,263,384.97","$945,335.46","$795,049.69","$563,363.84","$563,363.84","$563,363.84","$563,363.84","$563,363.84","$563,363.84","$563,363.84","$563,363.84","$563,363.84","$563,363.84"
2009,"$86,038.52","$1,307,603.45","$978,422.20","$822,876.43","$583,081.58","$583,081.58","$583,081.58","$583,081.58","$583,081.58","$583,081.58","$583,081.58","$583,081.58","$583,081.58","$583,081.58"
2010,"$89,049.87","$1,353,369.57","$1,012,666.98","$851,677.11","$603,489.43","$603,489.43","$603,489.43","$603,489.43","$603,489.43","$603,489.43","$603,489.43","$603,489.43","$603,489.43","$603,489.43"


We can also easily sum and collapse that aggregation, as it is a Stream`:

In [25]:
buildingAhotel['subtotal_egi'].sum()

date,Effective Gross Income for c0348c0f-a31f-421b-b6bc-0fe0d5ea947f [buildingAhotel] Aggregation (sum)
2001-12-31 00:00:00,"$6,854,263.68"
2002-12-31 00:00:00,"$7,094,162.91"
2003-12-31 00:00:00,"$7,342,458.61"
2004-12-31 00:00:00,"$7,599,444.66"
2005-12-31 00:00:00,"$7,865,425.23"
2006-12-31 00:00:00,"$8,140,715.11"
2007-12-31 00:00:00,"$8,425,640.14"
2008-12-31 00:00:00,"$8,720,537.54"
2009-12-31 00:00:00,"$9,025,756.36"
2010-12-31 00:00:00,"$9,341,657.83"


In [26]:
buildingAhotel['subtotal_egi'].sum().collapse()

date,Effective Gross Income for c0348c0f-a31f-421b-b6bc-0fe0d5ea947f [buildingAhotel] Aggregation (sum)
2011-12-31 00:00:00,"$90,078,677.93"


The above subtotals the Effective Gross Income (EGI) for the hotel space of
Building A over the hold period of the asset.

We can then do this for all `Entity`s in the graph, and plot the results:

In [27]:
sunburst = spatial_containment.sunburst(property='subtotal_egi')
treemap = spatial_containment.treemap(property='subtotal_egi')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]],
    subplot_titles=('Effective Gross Income - Sunburst Plot', 'Effective Gross Income - Treemap Plot'))
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='aggregate_egis_chart.html'

py.plot(fig, filename=f'assets/{filename}', auto_open=False)
IFrame(src=f'./{filename}', width='100%', height=800 * len(fig.data))

### Cost-generating Entities
Let's now do the same for the cost-generating `Entity`s. While it is routine to
register revenue-generating `Flow`s with `Entity`s that have lettable or
sellable floor space, the convention to also register all cost-generating
`Flow`s with only those same `Entity`s is possibly only done out of convenience
rather than accuracy.

For example, both operating and capital expenses may be attributed to building
systems (e.g. maintenance, upgrading, & replacement of MEP or facade
componentry) that are not coincident with the floor space they are servicing.
I.e., the convention to average (sub)total costs across (sub)total floor space
may be misleading, especially when attempting to compare different scenarios at
different resolutions (See {cite}`farevuu2018`, 5.4 Flaw of Averages).

By virtue of *Rangekeeper*'s multi-faceted approach to its object model, costs
can be attributed and aggregated to `Entity`s that are not spatially coincident
with the revenue-generating `Entity`s; in the example design, we will
use the 'services' relationship-type to register and aggregate some of these,
while space-specific costs will be registered and aggregated via spatial
containment:

#### Operational and Capital Expenses of Building Utilities & Systems
Of course, in a simplified model and without knowing design and engineering
details for the property, costs for operational and capital expenses will be
calculated from some proportional rate of spatial characteristics of the
building. In this example, we will use the following as simplifications of how
these types of costs may be calculated. Note that this is more a demonstration
of methodology, and in practice would adjust to real-world data about the
specific building systems and their costs.
-  'Floorplate-derived' Operational and Capital Expenses will be calculated as a
function of the subtotal floorplate area of 'building' `Entity`s. One could
think of this as representing the cost of operating the floor's MEP, or
maintaining, repairing and upgrading elements on the floors themselves
(e.g. FF&E)
- 'Facade-derived' Operational Expenses will be calculated as a function of the
  surface area of 'building' `Entity`. This would represent the cost of facade
maintenance, repair or replacement.
- 'Utility-derived' Capital Expenses will be calculated as a function of the
volume of 'utility' `Entity`s. This represents the cost of repairing and
upgrading the building's MEP and transportation systems.


#### Cost `Flow`s per `Entity`
There are some properties of `Entity`s that we need to produce before we can
calculate `Flow`s off them. In this case we need to produce the facade area
for any `Entity`s that have 'perimeter' and 'ftf' (Floor-to-Floor height)
properties:


In [28]:
# Aggregate Facade Areas:
for (entityId, entity) in property.get_entities().items():
    if hasattr(entity, 'perimeter') & hasattr(entity, 'ftf'):
        entity['facade_area'] = entity['perimeter'] * entity['ftf']
spatial_containment.aggregate(
    property='facade_area',
    label='subtotal_facade_area')

We now set up operational and capital costing parameters that are based off
either areas or volume, as well as growth rates:

In [29]:
floor_opex_per_area_pa = dict(
    hotel=-175,
    retail=-225,
    residential=-125,
    parking=-65,
    )
facade_opex_per_area_pa = dict(
    hotel=-50,
    retail=-100,
    residential=-50,
    parking=-25,
    )
opex_growth_rate = 0.035
floor_capex_per_area_pa = dict(
    hotel=-100,
    retail=-150,
    residential=-75,
    parking=-50,
    )
cores_capex_per_vol_pa = -100
plant_capex_per_vol_pa = -1000
capex_growth_rate = 0.035

Now we can set up our `Costs` class to produce cost `Flow`s and `Stream`s:

In [30]:
class Costs:
    def __init__(
            self,
            params: dict,
            spans: Spans):
        self.params = params
        self.spans = spans

In [31]:
@rk.update_class(Costs)
class Costs:
    def generate_space_costs(self):
        floor_opex = rk.flux.Flow.from_projection(
            name='Floor-derived Operating Expenses',
            value=self.params['initial_floor_opex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['opex_growth_rate']),
            sequence=self.spans.calc_span.to_sequence(frequency=self.params['frequency'])),
            units=currency.units)
        facade_opex = rk.flux.Flow.from_projection(
            name='Facade-derived Operating Expenses',
            value=self.params['initial_facade_opex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['opex_growth_rate']),
            sequence=self.spans.calc_span.to_sequence(frequency=self.params['frequency'])),
            units=currency.units)
        self.opex = rk.flux.Stream(
            name='Space Operating Expenses',
            flows=[floor_opex, facade_opex],
            frequency=self.params['frequency'])

        floor_capex = rk.flux.Flow.from_projection(
            name='Floor-derived Capital Expenses',
            value=self.params['initial_floor_capex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['capex_growth_rate']),
            sequence=self.spans.calc_span.to_sequence(frequency=rk.duration.Type.SEMIDECADE)),
            units=currency.units)
        self.floor_capex = floor_capex.trim_to_span(self.spans.calc_span)  # This is to avoid issues with using >yearly periodicity in projection

We will split the generation of space-based costs from the utility-based costs,
as they will be applied to different `Entity` types:

In [32]:
@rk.update_class(Costs)
class Costs:
    def generate_utils_costs(self):
        utils_capex = rk.flux.Flow.from_projection(
            name='Utilities-derived Capital Expenses',
            value=self.params['initial_utility_capex'],
            proj=rk.projection.Extrapolation(
                form=rk.extrapolation.Compounding(
                    rate=self.params['capex_growth_rate']),
                sequence=self.spans.calc_span.to_sequence(frequency=rk.duration.Type.SEMIDECADE)),
            units=currency.units)
        self.utils_capex = utils_capex.trim_to_span(self.spans.calc_span)  # This is to avoid issues with using >yearly periodicity in projection

Now we can run the `Costs` class against 'floor' and 'utilities' `Entity`s:

In [33]:
for floor in floors:
    floor_params = params.copy()

    floor_params['initial_floor_opex'] = floor_opex_per_area_pa[floor['use']] * floor['subtotal_gfa']
    floor_params['initial_facade_opex'] = facade_opex_per_area_pa[floor['use']] * floor['subtotal_facade_area']
    floor_params['initial_floor_capex'] = floor_capex_per_area_pa[floor['use']] * floor['subtotal_gfa']
    floor_params['opex_growth_rate'] = opex_growth_rate
    floor_params['capex_growth_rate'] = capex_growth_rate
    floor['params'] = floor_params

    floor['events'] = {} if not hasattr(floor, 'events') else floor['events']

    costs = Costs(
        params=floor_params,
        spans=spans)
    costs.generate_space_costs()

    floor['opex'] = costs.opex
    floor['capex'] = costs.floor_capex

The utilities are a bit different, as (for this example), their capex is a
factor of their volume. We can see that the utilities are the cores and plant
of the property:

In [39]:
IFrame("https://app.speckle.systems/projects/c0f66c35e3/models/119ad04487#savedView%3D%7B%2522id%2522%3A%2522b8d61129f8%2522%7D", width='100%', height=800)

In [40]:
utilities = property.filter_by_type(entity_type='utilities', is_assembly=False)

In [41]:
for utility in utilities.get_entities().values():
    utility_params = params.copy()

    if 'plant' in utility['name']:
        utility_params['initial_utility_capex'] = plant_capex_per_vol_pa * utility['volume']
    elif 'cores' in utility['name']:
        utility_params['initial_utility_capex'] = cores_capex_per_vol_pa * utility['volume']
    utility_params['capex_growth_rate'] = capex_growth_rate
    utility['params'] = utility_params

    utility['events'] = {} if not hasattr(utility, 'events') else utility['events']

    costs = Costs(
        params=utility_params,
        spans=spans)
    costs.generate_utils_costs()

    utility['capex'] = costs.utils_capex

In [42]:
list(utilities.get_entities().values())[0]['capex']

,Utilities-derived Capital Expenses
2005-12-31 00:00:00,"-$1,108,618.08"
2010-12-31 00:00:00,"-$1,147,419.71"


And finally aggregate them:

In [43]:
def aggregate_opex(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Operational Expenses',
        frequency=frequency)
property.aggregate(
    function=aggregate_opex,
    property='opex',
    label='subtotal_opex',
    relationship_type='spatiallyContains')

In [44]:
def aggregate_capex(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Capital Expenses',
        frequency=frequency)
property.aggregate(
    function=aggregate_capex,
    property='capex',
    label='subtotal_capex',
    relationship_type='spatiallyContains')

If we wish to see the ultimate aggregation of those costs, we can do so by
totalling them at their root `Entity`:

In [45]:
property_opex = property.get_roots()['spatiallyContains'][0]['subtotal_opex'].sum()
property_capex = property.get_roots()['spatiallyContains'][0]['subtotal_capex'].sum()

In [46]:
print('Property Average Periodic OPEX: ${:,.2f}'.format(property_opex.movements.mean()))
print('Property Average Periodic CAPEX: ${:,.2f}'.format(property_capex.movements.mean()))

Property Average Periodic OPEX: $-7,488,734.31
Property Average Periodic CAPEX: $-2,007,558.91


To get a sense check if this is reasonable, we can compare the opex and capex
to the PGI. First let's aggregate the PGI for all `Entity`s:

In [47]:
def aggregate_pgis(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Potential Gross Income',
        frequency=frequency)

In [48]:
spatial_containment.aggregate(
    property='pgi',
    function=aggregate_pgis,
    label='subtotal_pgi')

And total it to the root `Entity`:

In [49]:
property_pgi = property.get_roots()['spatiallyContains'][0]['subtotal_pgi'].sum()
print('Property Average Periodic PGI: ${:,.2f}'.format(property_pgi.movements.mean()))

Property Average Periodic PGI: $18,918,012.54


Now we can calculate the Expenses ratios:

In [50]:
print('Property OpEx as proportion of PGI: {:.2%}'.format(-property_opex.movements.mean() / property_pgi.movements.mean()))

Property OpEx as proportion of PGI: 39.59%


In [51]:
print('Property CapEx as proportion of PGI: {:.2%}'.format(-property_capex.movements.mean() / property_pgi.movements.mean()))

Property CapEx as proportion of PGI: 10.61%


Looking at just the cost-producing `Entity`s, we can see that in the OpEx and
CapEx breakdowns, there is a significant amount of cost generated by non-floor
(i.e., non-revenue-generating) `Entity`s, like cores and plant. This type of
analysis is important in understanding the cost drivers of a property, and
where the most effective cost-saving measures could be applied.

In [52]:
opex = spatial_containment.sunburst(property='subtotal_opex')
capex = spatial_containment.sunburst(property='subtotal_capex')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "sunburst"}]],
    subplot_titles=('Total Operational Expenses', 'Total Capital Expenses'))
fig.append_trace(opex, row=1, col=1)
fig.append_trace(capex, row=2, col=1)
filename='aggregate_exp_chart.html'

py.plot(fig, filename=f'assets/{filename}', auto_open=False)
IFrame(src=f'./{filename}', width='100%', height=800 * len(fig.data))

## Calculating Overall Valuation
Now that we have our revenues and cost line items (`Flow`s) generated per
`Entity`, we can accumulate them into Net Operating Income (NOI) and
Net Annual Cashflow (NACF) subtotals, as well as the reversion (disposition, or
sale) of the whole asset, in order to determine the overall valuation of the
property.

We can also calculate the net position of each `Entity` as its 'Net Annual
Cashflow (NACF)':

In [53]:
for entity in property.get_entities().values():
    noi = [getattr(entity, 'egi', None), getattr(entity, 'opex', None)]
    noi = [flow.sum() if isinstance(flow, rk.flux.Stream) else flow for flow in list(filter(None, noi))]
    if len(noi) > 0:
        entity['noi'] = rk.flux.Stream(
            name='Net Operating Income',
            flows=noi,
            frequency=frequency)
        capex = getattr(entity, 'capex', None)
        if capex is not None:
            entity['nacf'] = rk.flux.Stream(
                name='Net Annual Cashflow',
                flows=entity['noi'].flows + [capex],
                frequency=frequency)

In [54]:
def aggregate_noi(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Net Operating Income',
        frequency=frequency)
property.aggregate(
    function=aggregate_noi,
    property='noi',
    label='subtotal_noi',
    relationship_type='spatiallyContains')

In [55]:
def aggregate_nacf(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Net Annual Cashflow',
        frequency=frequency)
property.aggregate(
    function=aggregate_nacf,
    property='nacf',
    label='subtotal_nacf',
    relationship_type='spatiallyContains')

### Reversion
To calculate the reversion cashflow, we set up a period that spans the 10th year
of the project. This only applies to the asset as a whole, so it is recorded
at the root `Entity`:

In [56]:
root = property.get_roots()['spatiallyContains'][0]

In [59]:
reversion_span = rk.duration.Span.from_duration(
    name='Reversion',
    date=params['start_date'] + pd.DateOffset(years=9),
    duration=params['frequency'])

exit_caprate = 0.05
reversion_flow = rk.flux.Flow.from_projection(
    name='Reversion',
    value=root['subtotal_nacf'].sum().movements.values[-1] / exit_caprate,
    proj=rk.projection.Distribution(
        form=rk.distribution.Uniform(),
        sequence=reversion_span.to_sequence(frequency=params['frequency'])),
    units=currency.units)
root['reversion'] = reversion_flow
reversion_flow

,Reversion
2010-12-31 00:00:00,"$284,674,637.71"


Now we can aggregate the net and reversion cashflows in the root `Entity` in
order to derive the project's complete cashflows:

In [61]:
net_cashflows_with_reversion = (rk.flux.Stream(
    name='Net Cashflow with Reversion',
    flows=[root['subtotal_nacf'].sum(), root['reversion']],
    frequency=rk.duration.Type.YEAR)
        .trim_to_span(
            rk.duration.Span(
                start_date=params['start_date'],
                end_date=reversion_span.end_date)
    )
)
root['nacf_reversion'] = net_cashflows_with_reversion
net_cashflows_with_reversion

,Net Annual Cashflow for afe8e730-ad0f-4c2a-a15c-e249e4518fa8 [property] Aggregation (sum),Reversion
2001,"$8,134,691.89",0
2002,"$8,599,446.40",0
2003,"$9,091,532.28",0
2004,"$9,612,601.91",0
2005,"$6,862,704.94",0
2006,"$10,748,821.65",0
2007,"$11,367,816.34",0
2008,"$12,023,498.34",0
2009,"$12,718,103.14",0
2010,"$10,036,741.15","$284,674,637.71"


In [62]:
table = rk.flux.Stream(
    name='Table 1.1',
    flows=[
        root['subtotal_egi'].sum(),
        root['subtotal_opex'].sum(),
        root['subtotal_noi'].sum(),
        root['subtotal_capex'].sum(),
        root['subtotal_nacf'].sum(),
        root['reversion'],
        root['nacf_reversion'].sum()
        ],
    frequency=rk.duration.Type.YEAR
    )
table

,Effective Gross Income for afe8e730-ad0f-4c2a-a15c-e249e4518fa8 [property] Aggregation (sum),Operational Expenses for afe8e730-ad0f-4c2a-a15c-e249e4518fa8 [property] Aggregation (sum),Net Operating Income for afe8e730-ad0f-4c2a-a15c-e249e4518fa8 [property] Aggregation (sum),Capital Expenses for afe8e730-ad0f-4c2a-a15c-e249e4518fa8 [property] Aggregation (sum),Net Annual Cashflow for afe8e730-ad0f-4c2a-a15c-e249e4518fa8 [property] Aggregation (sum),Reversion,Net Cashflow with Reversion (sum)
2001,"$14,402,849.55","-$6,268,157.66","$8,134,691.89",0,"$8,134,691.89",0,"$8,134,691.89"
2002,"$15,086,989.58","-$6,487,543.18","$8,599,446.40",0,"$8,599,446.40",0,"$8,599,446.40"
2003,"$15,806,139.47","-$6,714,607.19","$9,091,532.28",0,"$9,091,532.28",0,"$9,091,532.28"
2004,"$16,562,220.35","-$6,949,618.44","$9,612,601.91",0,"$9,612,601.91",0,"$9,612,601.91"
2005,"$17,357,265.38","-$7,192,855.09","$10,164,410.29","-$5,919,092.61","$6,862,704.94",0,"$6,862,704.94"
2006,"$18,193,426.66","-$7,444,605.02","$10,748,821.65",$0.00,"$10,748,821.65",0,"$10,748,821.65"
2007,"$19,072,982.53","-$7,705,166.19","$11,367,816.34",$0.00,"$11,367,816.34",0,"$11,367,816.34"
2008,"$19,998,345.35","-$7,974,847.01","$12,023,498.34",$0.00,"$12,023,498.34",0,"$12,023,498.34"
2009,"$20,972,069.80","-$8,253,966.66","$12,718,103.14",$0.00,"$12,718,103.14",0,"$12,718,103.14"
2010,"$21,996,861.68","-$8,542,855.49","$13,454,006.19","-$6,126,260.85","$10,036,741.15","$284,674,637.71","$294,711,378.86"


### Valuation
With our net cashflows including reversion, if we specify a discount rate, we
can calculate the present value of the property:

In [63]:
discount_rate = 0.07
pvs = root['nacf_reversion'].sum().pv(
    name='Present Value',
    frequency=rk.duration.Type.YEAR,
    rate=discount_rate)
pvs

,Present Value
2001-12-31 00:00:00,"$7,602,515.78"
2002-12-31 00:00:00,"$7,511,089.53"
2003-12-31 00:00:00,"$7,421,398.50"
2004-12-31 00:00:00,"$7,333,407.97"
2005-12-31 00:00:00,"$4,893,013.78"
2006-12-31 00:00:00,"$7,162,393.72"
2007-12-31 00:00:00,"$7,079,304.69"
2008-12-31 00:00:00,"$6,997,785.50"
2009-12-31 00:00:00,"$6,917,805.44"
2010-12-31 00:00:00,"$149,816,320.83"


In [64]:
property_pv = pvs.collapse().movements.item()
print('Property PV: ${:,.0f}'.format(property_pv))

Property PV: $212,735,036
